In [1]:
# Code to get all the revised LC points (from rerunning EITfluxsum.py for subsets of the years 1996-2022) into the same results file
# and all the metadata files into the same metadata file

# also revise the EIT pinhole flag to only apply to images which have filter = 'Clear'

In [1]:
import EITfunc.EITlook as eit

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clrs
import matplotlib.cm as cmx
import matplotlib.colorbar as cb
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import gridspec

import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

import os

from astropy import units as u
from astropy.constants import G
from astropy.io import ascii, fits
from astropy.table import Table, vstack
from astropy.timeseries import LombScargle
import astropy.time
from astropy.coordinates import HeliocentricMeanEcliptic, SkyCoord

import sunpy.coordinates

import scipy.optimize as sciop
from scipy.stats import mode, binned_statistic

import time
import copy

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
ls ./EIT/EITresults

EIT_LC_1996_1999.txt              EIT_headerMetadata_1999_2000.txt
EIT_LC_1999_2000.txt              EIT_headerMetadata_2000_2003.txt
EIT_LC_2000_2003.txt              EIT_headerMetadata_2003_2005.txt
EIT_LC_2003_2005.txt              EIT_headerMetadata_2005_2019.txt
EIT_LC_2005_2019.txt              EIT_headerMetadata_2019_2023.txt
EIT_LC_2019_2023.txt              EITdark.py*
EIT_LC_withflags.txt              EITfluxsum.py*
EIT_headerMetadata_1996_1999.txt  EITmetadata.py


In [3]:
start_years = np.array((1996, 1999, 2000, 2003, 2005, 2019))
end_years = np.array((1999,2000,2003,2005,2019,2023))

In [4]:
LCheader = 't_obs[JD] solar_distance[m] wavelength[angstrom] unnorm_flux[DN/s] unnorm_flux_unc[DN/s] pinholes[1ifYes] cameraError[1ifYes]'
metadataHeader = 'DATE-BEG DATE-AVG XPOSURE CMDXPOS SHUTCLS FILTER WAVELNTH OBJECT SCI_OBJ OBS_PROG CMP_NO UCD EXPMODE LYNESYNC ROUTPORT NLEBPROC LEBPROC1 LEBPROC2 LEBPROC3 WCSNAME CTYPE1 CTYPE2 CUNIT1 CUNIT2 PC1_1 PC1_2 PC2_1 PC2_2 CDELT1 CDELT2 CRVAL1 CRVAL2 CRPIX1 CRPIX2 CROTA CFTEMP CCDTEMP SOLAR_B0 RSUN_ARC RSUN_OBS RSUN_REF CAR_ROT DSUN_OBS SC_X0 SC_Y0 SC_ROLL HAEX_OBS HAEY_OBS HAEZ_OBS'

In [9]:
allLC = np.zeros((1,7))
allmetadata = Table.read("./EIT/EITresults/EIT_headerMetadata_1996_1999.txt",format='ascii')
print(len(allmetadata))

for i in range(len(start_years)):
    start_yr = start_years[i]
    end_yr = end_years[i]

    LCdata = np.genfromtxt("./EIT/EITresults/EIT_LC_{0}_{1}.txt".format(start_yr,end_yr))
    print(np.shape(LCdata))
    allLC = np.vstack((allLC, LCdata))

    if i > 0:
        metadata = Table.read("./EIT/EITresults/EIT_headerMetadata_{0}_{1}.txt".format(start_yr,end_yr),format='ascii')
        print(len(metadata))
        metadata['CMP_NO'] = metadata['CMP_NO'].astype(str)
        allmetadata = vstack((allmetadata,metadata))

allLC = allLC[1:]
print(np.shape(allLC))
print(len(allmetadata))

# sort by observation time
t_idxs = np.argsort(allLC[:,0])
allLC = allLC[t_idxs]

t_idxs_metadata = np.argsort(allmetadata['DATE-BEG'])
allmetadata = allmetadata[t_idxs_metadata]
print(allmetadata)

9641
(9641, 7)
(4372, 7)
4372
(14787, 7)
14787
(8645, 7)
8645
(44477, 7)
44477
(9961, 7)
9961
(91883, 7)
91883
     DATE-BEG           DATE-AVG      XPOSURE ...    HAEY_OBS      HAEZ_OBS 
------------------ ------------------ ------- ... -------------- -----------
 2450098.509462373 2450098.5129467477   602.1 ... 130562576000.0 -77407740.0
2450098.5201983913  2450098.520678391  82.944 ... 130562576000.0 -77407740.0
2450098.5366403703  2450098.536677118    6.35 ... 130562576000.0 -77407740.0
  2450099.07985044  2450099.079920463    12.1 ... 130562576000.0 -77407740.0
2450102.2426729975 2450102.2427430553    12.1 ... 126939368000.0 -67100020.0
 2450102.278504838  2450102.278574815    12.1 ... 126939368000.0 -67100020.0
 2450102.401938831 2450102.4025296876   102.1 ... 126939368000.0 -67100020.0
2450103.1874925694  2450103.187562593    12.1 ... 125652736000.0 -63552880.0
 2450103.212478449  2450103.212548426    12.1 ... 125652736000.0 -63552880.0
2450103.2389932405  2450103.240741505   30

In [10]:
np.allclose(allLC[:,0], allmetadata['DATE-BEG'], 1.e-12)
print(allLC[:,0])
print(allmetadata['DATE-BEG'])

[2450098.509462 2450098.520198 2450098.53664  ... 2459945.045995
 2459945.051343 2459945.055437]
     DATE-BEG     
------------------
 2450098.509462373
2450098.5201983913
2450098.5366403703
  2450099.07985044
2450102.2426729975
 2450102.278504838
 2450102.401938831
2450103.1874925694
 2450103.212478449
2450103.2389932405
               ...
 2459944.044638472
2459944.0499038887
2459944.0539360186
2459944.5416033333
2459944.5510996296
2459944.5551944096
2459945.0418584724
2459945.0459949537
 2459945.051343287
2459945.0554369907
Length = 91883 rows


In [11]:
print(np.unique(allmetadata['FILTER']))

#print(len(allmetadata[allmetadata['FILTER'] == 'Al_+1']))
#print(len(allmetadata[allmetadata['FILTER'] == 'Al_sup']))
#print(len(allmetadata[allmetadata['FILTER'] == 'Clear']))


# number of pinholes
#print(len(allLC[allLC[:,5] == 0]))
print(len(allLC[allLC[:,5] == 1]))

Al_filterMask = (allmetadata['FILTER'] != 'Clear')
#print(len(allmetadata[~Al_filterMask]))


#get rid of spurious detections of pinholes where there can't have been any because the filter was in place
allLC[:,5][Al_filterMask] = 0

print(len(allLC[allLC[:,5] == 1]))


FILTER
------
 Al_+1
Al_sup
 Clear
10424
1495


In [12]:
#pinholes = allLC[:,5]

Al_filterMask = (allmetadata['FILTER'] == 'Al_+1') | (allmetadata['FILTER'] == 'Al_sup')
print(len(allmetadata[~Al_filterMask]))

#get rid of spurious detections of pinholes where there can't have been any because the filter was in place
allLC[:,5][Al_filterMask] = 0

print(len(allLC[allLC[:,5] == 1]))


6083
1495


In [13]:
np.savetxt("./EIT/EITresults/EIT_LC_withflags.txt", allLC, fmt="%f", header=LCheader)
ascii.write(allmetadata, "./EIT/EITresults/EIT_headerMetadata.txt")